**!** *Tip: type `shift+enter` OR click the ` >| Run ` button above to exectue a cell*

### Init

In [1]:
from models import *
from processing import *

### Setup

In [2]:
coil = Coil(wired_in=Coil.SERIES, current=Coil.DC, mag_Br=1.4)

In [3]:
tube = Tube(OD=21, ID=23, effective_length=150, unit="mm")

In [4]:
p = Process(coil, tube)

### Exploring Parameters

#### wire diameter and coil height

Explore against one or multiple parameters (ctrl/Command+click, or shift+click)

In [ ]:
p.explore_wire_diameter(resolution=0.001, debug=False)

#### After exploring, choose the parameters

In [ ]:
coil.set_wire_d(1.0, unit="mm") 
coil.set_layer_count(10)  # comment if you want to explore graph
coil.mag_Br = 1.5  #  N42 neodynium magnet

Once we have explored values for wire diameter and current, 
we can explore force against extension of plunger and layer height

### Explore Actuator Force

In [ ]:
p.explore_actuator_force()

### Playground

In [ ]:
coil.set_wire_d(1.0, unit='mm')

coil.set_layer_count(14)
# OR
#  coil.set_layer_height(14.0, unit='mm')
#  this will automatically calculate layer count

Electrical Properties

In [ ]:
I, R = coil.get_current(tube), coil.get_resistance(tube); V = I * R
p.out(u"V=%0.2fV \n"
       "I=%0.2fA \n"
       "R=%0.2f\u2126 \n"
       "P=%0.2fW" % (V, I, R, V*I))

Physical Properties

In [ ]:
wire_d, coil_m, coil_L = coil.get_wire_d(unit='mm'), \
                         coil.get_coil_mass(tube, unit='kg'), \
                         coil.get_coil_length(tube, unit='m')
p.out("wire diameter=%.2fmm \n"
      "coil mass=%.2fkg \n"
      "coil length=%.1fm"% (wire_d, coil_m, coil_L))

Force of actuator, represented as the vector (magnitude, direction) in the axis of travel.

In [ ]:
samples = 11  # how many data points over the stroke of the actuator
for z in [(x/(samples-1))*tube.effective_length for x in range(0, samples, 1)]:
    print("x=%.3fm," % z, end=' F=')
    coil_layers = 14
    Bmag = 1.241  #  N38 Neodydlium magnet
#     Bmag = 1.4  #  N42 Neodydlium magnet
    Bsol_z = coil.calculate_magnetic_field(tube, coil_layers=coil_layers, z=z)

    d = tube.OD + coil_layers*coil.wire_d/2
    A = math.pi*pow(d/2 + (tube.OD - tube.ID)/2, 2)
    mu = 4.0 * math.pi * pow(10, -7)

    f_z = math.sqrt(pow(Bsol_z[0], 2) + pow(Bsol_z[1], 2)) * Bmag * A / mu
    print("%.3f" % (math.sqrt(pow(Bsol_z[0], 2) + pow(Bsol_z[1], 2)) * Bmag * A / mu), end=' ')  # resolved
    print("(%.3f, %.3f)" % (Bsol_z[0] * Bmag * A / mu, Bsol_z[1] * Bmag * A / mu))  # vector